In [1]:
import torch
import datasets
import os
import aqlm
import time
import numpy as np
import sys
import datasets
import pickle

In [ ]:
for name, p in model.named_parameters():
	

In [2]:
sys.path.append("d:\\projects\\affect\\TUCORE-GCN\\tucore_gcn_transformers")
sys.path.append("d:\\projects\\affect\\TUCORE-GCN\\")

In [3]:
tucore_data = datasets.load_from_disk("./datasets/DialogRE/parity_2/")
tucore_data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'label_ids', 'input_ids', 'input_mask', 'segment_ids', 'speaker_ids', 'mention_ids', 'turn_masks', 'graph', 'graph_data'],
        num_rows: 5997
    })
    test: Dataset({
        features: ['tokens', 'label_ids', 'input_ids', 'input_mask', 'segment_ids', 'speaker_ids', 'mention_ids', 'turn_masks', 'graph', 'graph_data'],
        num_rows: 1862
    })
    validation: Dataset({
        features: ['tokens', 'label_ids', 'input_ids', 'input_mask', 'segment_ids', 'speaker_ids', 'mention_ids', 'turn_masks', 'graph', 'graph_data'],
        num_rows: 1914
    })
})

In [3]:
from bertflashtest2 import TUCOREGCN_BertForSequenceClassification, TUCOREGCN_BertConfig
cfg = TUCOREGCN_BertConfig.from_json_file("./models/BERT/tucoregcn_bert_mlc.json")
b = TUCOREGCN_BertForSequenceClassification.from_pretrained("bert-base-uncased", config=cfg)
b = b.to(torch.float16).cuda()

In [4]:
b

TUCOREGCN_BertForSequenceClassification(
  (tucoregcn_bert): TUCOREGCN_Bert(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (speaker_embeddings): Embedding(512, 768)
      )
      (emb_drop): Dropout(p=0.1, inplace=False)
      (emb_ln): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (encoder): BertEncoder(
        (layers): ModuleList(
          (0-11): 12 x Block(
            (mixer): MHA(
              (Wqkv): FusedDense(in_features=768, out_features=2304, bias=True)
              (inner_attn): FlashSelfAttention(
                (drop): Dropout(p=0.1, inplace=False)
              )
              (inner_cross_attn): FlashCrossAttention(
                (drop): Dropout(p=0.1, inplace=False)
              )
              (out_proj): FusedDense(in_features=768, out_features=768, bias=True)


In [5]:
for n, p in b.named_parameters():
	print(n, p.requires_grad)

tucoregcn_bert.bert.embeddings.word_embeddings.weight True
tucoregcn_bert.bert.embeddings.position_embeddings.weight True
tucoregcn_bert.bert.embeddings.token_type_embeddings.weight True
tucoregcn_bert.bert.embeddings.speaker_embeddings.weight True
tucoregcn_bert.bert.emb_ln.weight True
tucoregcn_bert.bert.emb_ln.bias True
tucoregcn_bert.bert.encoder.layers.0.mixer.Wqkv.weight True
tucoregcn_bert.bert.encoder.layers.0.mixer.Wqkv.bias True
tucoregcn_bert.bert.encoder.layers.0.mixer.out_proj.weight True
tucoregcn_bert.bert.encoder.layers.0.mixer.out_proj.bias True
tucoregcn_bert.bert.encoder.layers.0.norm1.weight True
tucoregcn_bert.bert.encoder.layers.0.norm1.bias True
tucoregcn_bert.bert.encoder.layers.0.mlp.fc1.weight True
tucoregcn_bert.bert.encoder.layers.0.mlp.fc1.bias True
tucoregcn_bert.bert.encoder.layers.0.mlp.fc2.weight True
tucoregcn_bert.bert.encoder.layers.0.mlp.fc2.bias True
tucoregcn_bert.bert.encoder.layers.0.norm2.weight True
tucoregcn_bert.bert.encoder.layers.0.norm2.b

In [5]:
shard = tucore_data['train'][0:32]
device = "cuda"
label_ids = torch.LongTensor(shard["label_ids"]).contiguous().float().to(device)
input_ids = torch.LongTensor(shard["input_ids"]).contiguous().to(device)
segment_ids = torch.LongTensor(shard["segment_ids"]).contiguous().to(device)
input_masks = torch.LongTensor(shard["input_mask"]).contiguous().to(device)
mention_ids = torch.LongTensor(shard["mention_ids"]).contiguous().to(device)
speaker_ids = torch.LongTensor(shard["speaker_ids"]).contiguous().to(device)
turn_mask = torch.LongTensor(shard["turn_masks"]).contiguous().to(torch.float16).to(device)
# forgot to flatten the list 1
graphs = [pickle.loads(g)[0].to(device) for g in shard['graph']]
#print(shard["tokens"])
#print(graphs)

In [6]:
out = b(input_ids=input_ids, token_type_ids=segment_ids, attention_mask=input_masks, speaker_ids=speaker_ids, graphs=graphs, mention_ids=mention_ids, turn_mask=turn_mask, labels=label_ids)
out

torch.Size([32, 12, 512, 64]) torch.Size([32, 1, 512, 512])


SequenceClassifierOutput(loss=tensor(1.2008, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 2.0977, -4.8203,  2.6484,  ..., -3.0352, -1.8633,  2.3730],
        [-1.5371, -0.7388, -2.7207,  ..., -2.2441,  1.3584, -1.2959],
        [ 0.0704, -3.6660,  0.5093,  ...,  0.1266,  1.3408, -0.2920],
        ...,
        [ 0.7241,  0.9243,  1.0332,  ..., -2.3984, -0.9561,  0.3440],
        [ 2.1250, -0.6787,  0.4321,  ..., -4.2852,  2.3691,  0.3171],
        [ 2.4395, -5.1055,  1.6514,  ..., -0.4211,  3.2871,  4.8672]],
       device='cuda:0', dtype=torch.float16, grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [9]:
out.loss.backward()

In [9]:
from tucore_gcn_bert_modelling import TUCOREGCN_BertForSequenceClassification as OldTUCORE, TUCOREGCN_BertConfig as OldTUCOREConfig
from transformers.models.bert.modeling_bert import BertAttention
cfg_old = OldTUCOREConfig.from_json_file("./models/BERT/tucoregcn_bert_mlc.json")
b_old = OldTUCORE.from_pretrained("bert-base-uncased", config=cfg).cuda()


Some weights of TUCOREGCN_BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.classifier.bias', 'bert.classifier.weight', 'bert.tucoregcn_bert.GCN_layers.0.h_bias', 'bert.tucoregcn_bert.GCN_layers.0.loop_weight', 'bert.tucoregcn_bert.GCN_layers.0.weight', 'bert.tucoregcn_bert.GCN_layers.1.h_bias', 'bert.tucoregcn_bert.GCN_layers.1.loop_weight', 'bert.tucoregcn_bert.GCN_layers.1.weight', 'bert.tucoregcn_bert.LSTM_layers.0.bilstm2hiddnesize.bias', 'bert.tucoregcn_bert.LSTM_layers.0.bilstm2hiddnesize.weight', 'bert.tucoregcn_bert.LSTM_layers.0.lstm.bias_hh_l0', 'bert.tucoregcn_bert.LSTM_layers.0.lstm.bias_hh_l0_reverse', 'bert.tucoregcn_bert.LSTM_layers.0.lstm.bias_hh_l1', 'bert.tucoregcn_bert.LSTM_layers.0.lstm.bias_hh_l1_reverse', 'bert.tucoregcn_bert.LSTM_layers.0.lstm.bias_ih_l0', 'bert.tucoregcn_bert.LSTM_layers.0.lstm.bias_ih_l0_reverse', 'bert.tucoregcn_bert.LSTM_layers.0.lstm.bias_ih_l1', 'bert.tucoreg

In [10]:
out_old = b_old(input_ids=input_ids, token_type_ids=segment_ids, attention_mask=input_masks, speaker_ids=speaker_ids, graphs=graphs, mention_ids=mention_ids, turn_mask=turn_mask, labels=label_ids)
out_old

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


torch.Size([1, 12, 512, 512])


SequenceClassifierOutput(loss=tensor(0.7859, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.3151,  0.0591, -0.2975,  0.0018, -0.7019, -0.2182,  0.2195,  0.5404,
          0.3145,  1.0241,  0.2763, -0.9889,  0.0384, -1.3534,  0.9168,  1.1087,
         -0.8001,  1.0619,  1.4883, -1.1176, -0.4786, -0.4132, -0.4819, -0.3732,
          0.0739,  0.4452, -0.2933, -0.8128,  0.1401, -0.2243,  1.3199, -0.8920,
          0.7906,  1.4601, -0.0192, -0.0884]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=tensor([[[[1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0783, 0.1116,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0900, 0.1012,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 1.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.0000]],

        

In [12]:
o = b.bert(
            input_ids,
            position_ids = None,
            speaker_ids=speaker_ids,
            token_type_ids=segment_ids,
            attention_mask=input_masks,)


In [17]:
o.last_hidden_state

tensor([[[-0.0575, -0.6606,  0.3245,  ..., -1.3594,  1.3398, -2.0703],
         [ 0.1743,  0.2061, -0.6011,  ..., -0.6396,  1.6611, -0.7979],
         [ 1.0068, -0.9053, -0.0556,  ..., -1.1133,  0.2900, -1.3008],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<ViewBackward0>)

In [ ]:
torch.Size([1, 512, 768])

In [18]:
turn_mask

tensor([[[1, 0, 0,  ..., 0, 0, 0],
         [0, 1, 1,  ..., 0, 0, 0],
         [0, 1, 1,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 1, 0, 0],
         [0, 0, 0,  ..., 0, 1, 0],
         [0, 0, 0,  ..., 0, 0, 1]]], device='cuda:0')

In [3]:
from bertflashtest2 import BertModel, TUCOREGCN_BertConfig
cfg = TUCOREGCN_BertConfig.from_json_file("./models/BERT/tucoregcn_bert_mlc.json")
b = BertModel.from_pretrained("bert-base-uncased", config=cfg)
b = b.cuda().to(torch.float16)

In [4]:
b(input_ids=torch.tensor([[242,411,4253, 4], [1,1,4, 4]], device='cuda', dtype=torch.long), speaker_ids=torch.tensor([[1,1,1,0], [1,1,1,0]], device='cuda', dtype=torch.long))

None
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])


BaseModelOutputWithPoolingAndNoAttention(last_hidden_state=tensor([[[-1.0557,  0.1991, -0.9048,  ...,  0.8823, -1.6025, -0.3289],
         [-1.1992,  0.2029, -1.1611,  ...,  1.0039, -1.1309,  0.7852],
         [-0.1595,  0.1234, -0.3401,  ...,  0.6060, -1.0635,  0.3459],
         [ 0.6133, -0.2169, -0.1602,  ...,  0.6953, -1.4170,  0.0222]],

        [[ 0.2034,  0.4106, -0.1943,  ...,  0.0327, -0.6909,  0.5894],
         [ 0.3235,  0.2522, -0.0910,  ...,  0.6362, -1.7666,  1.6436],
         [ 0.7251,  0.2554,  0.1146,  ..., -0.2432, -1.7236,  0.5312],
         [-0.2998,  0.9570,  0.6396,  ...,  0.4163, -1.9893,  0.0149]]],
       device='cuda:0', dtype=torch.float16,
       grad_fn=<DropoutAddLayerNormFnBackward>), pooler_output=tensor([[ 0.3389, -0.1530,  0.2493,  ...,  0.1101, -0.5947, -0.5073],
        [ 0.2275,  0.0264, -0.6108,  ..., -0.0265, -0.2347, -0.5386]],
       device='cuda:0', dtype=torch.float16, grad_fn=<TanhBackward0>), hidden_states=None)

In [3]:
from berttest import BertModel, BertConfig
cfg = BertConfig.from_json_file("./models/BERT/tucoregcn_bert_mlc.json")
b = BertModel.from_pretrained("bert-base-uncased", config=cfg)
b = b.cuda().to(torch.float16)

In [ ]:
b(input_ids=torch.tensor([[242,411,4253, 4], [1,1,4, 4]], device='cuda', dtype=torch.long), speaker_ids=torch.tensor([[1,1,1,0], [1,1,1,0]], device='cuda', dtype=torch.long))

In [3]:
from berttest import BertModel as FBertModel, BertConfig as FBertConfig
cfg = FBertConfig.from_json_file("./models/BERT/tucoregcn_bert_mlc.json")
b = FBertModel.from_pretrained("bert-base-uncased", config=cfg)
b = b.cuda().to(torch.float16)

In [ ]:
torch.Size([2, 4, 768])
torch.Size([0, 768])


In [4]:
b(input_ids=torch.tensor([[242,411,4253, 4], [1,1,4, 4]], device='cuda', dtype=torch.long))

torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])


BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.1273,  1.0635,  0.8647,  ...,  1.6279, -0.1172, -0.7095],
         [-1.3789,  2.6348, -0.3916,  ...,  2.0254,  0.5186,  0.9683],
         [ 1.0078,  0.1206,  0.2401,  ...,  1.5693,  0.1462, -0.5308],
         [ 0.4731,  1.6836,  0.8052,  ...,  1.3770, -0.3347, -1.5928]],

        [[ 0.3369,  0.4458,  0.3726,  ...,  0.7710, -0.9214, -0.5576],
         [-0.3430,  0.8477,  0.6670,  ...,  1.5322, -0.2281, -0.1907],
         [ 1.0498,  0.2827,  1.6172,  ...,  1.2080, -0.7671, -0.5688],
         [ 1.0684,  0.0049,  0.1741,  ...,  1.8379, -0.2996, -1.2695]]],
       device='cuda:0', dtype=torch.float16,
       grad_fn=<DropoutAddLayerNormFnBackward>), pooler_output=tensor([[ 0.3062,  0.4792,  0.3333,  ...,  0.4758,  0.6953, -0.1066],
        [-0.5225,  0.3008,  0.2153,  ...,  0.2267,  0.1681, -0.0210]],
       device='cuda:0', dtype=torch.float16, grad_fn=<TanhBackward0>), hidden_states=None, past_key_values=None, att

In [4]:
b(input_ids=torch.tensor([[242,411,4253, 4], [1,1,4, 4]], device='cuda', dtype=torch.long), speaker_ids=torch.tensor([[1,1,1,0], [1,1,1,0]], device='cuda', dtype=torch.long))

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.]], device='cuda:0')
torch.Size([8, 768])
torch.Size([8, 768])
tensor([[[ 0.1370, -0.6714, -0.3083,  ..., -0.1620, -0.5376, -0.4927],
         [ 0.2620,  0.2482,  0.4829,  ..., -0.3311,  0.2043, -0.1761],
         [-0.7661, -1.0352,  0.5298,  ...,  0.4292,  0.2485, -1.2930],
         ...,
         [ 0.4753, -0.0040,  0.0613,  ...,  0.3113,  0.6152,  0.3567],
         [ 0.7925, -0.7310, -0.2534,  ...,  0.9414, -0.9277, -0.5991],
         [ 0.2864,  0.2115, -0.1990,  ..., -0.1597, -0.9922,  0.8960]],

        [[-0.1299, -0.1632, -0.0262,  ..., -0.3716,  0.1635, -0.0753],
         [-1.0771, -0.2529, -1.2334,  ..., -0.1819,  0.8052, -0.0424],
         [-0.4263,  0.1910,  0.7476,  ..., -0.1449,  0.6797, -0.6064],
         ...,
         [ 0.0396, -0.4363,  0.6763,  ...,  1.5762,  0.7280,  0.0129],
         [ 0.5576, -0.7529,  0.0433,  ...,  0.6987, -0.5986, -0.6836],
         [ 0.1493,  0.1188,  0.0728,  ..., -0.1699, -0.0368,  0.2072]],

    

AttributeError: 'tuple' object has no attribute 'contiguous'

In [3]:
import sys
sys.path.append("d:\\projects\\affect\\TUCORE-GCN\\tucore_gcn_transformers")
sys.path.append("d:\\projects\\affect\\TUCORE-GCN\\")
from bertflashtest import BertModel, TUCOREGCN_BertConfig
cfgs = TUCOREGCN_BertConfig.from_json_file("./models/BERT/tucoregcn_bert_mlc.json")
cfgs

TUCOREGCN_BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "attention_probs_stochastic_dropout_prob": [
    0.0
  ],
  "classifier_dropout": null,
  "debug_ret": false,
  "dropout_cls": "pytorch_dropout",
  "fused_bias_fc": [
    true
  ],
  "fused_dropout_add_ln": [
    true
  ],
  "fused_mlp": [
    true
  ],
  "gcn_act": "relu",
  "gcn_dropout": 0.6,
  "gcn_layers": 2,
  "hidden_act": "gelu_fast",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "hidden_stochastic_dropout_prob": [
    0.0
  ],
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22"

In [3]:
import dropout_layer_norm

In [12]:
from flash_attn.modules.block import Block
b = Block(768, fused_dropout_add_ln=True).cuda()
b(torch.rand((1, 512, 768), device="cuda"))

(tensor([[[ 0.1205,  0.1024, -0.1764,  ...,  0.0771, -0.1893,  0.0387],
          [-0.2876, -0.0755,  0.0746,  ...,  0.0045,  0.0380, -0.0340],
          [ 0.0745, -0.1656, -0.0764,  ...,  0.0611, -0.1659, -0.1878],
          ...,
          [ 0.2771, -0.3058, -0.1100,  ..., -0.1033, -0.3371,  0.2852],
          [ 0.3683, -0.0591, -0.2646,  ...,  0.1142,  0.0963, -0.0448],
          [ 0.1025, -0.5471,  0.0992,  ..., -0.0914, -0.2076, -0.0386]]],
        device='cuda:0', grad_fn=<ViewBackward0>),
 tensor([[[ 0.7346,  0.7809,  0.4057,  ...,  0.1942,  0.6727, -0.0043],
          [ 0.2975,  0.9328,  0.4387,  ...,  0.6514,  0.0458,  0.7582],
          [ 0.5049,  0.7073,  0.3685,  ...,  0.0822,  0.8734,  0.8965],
          ...,
          [ 0.5644,  0.7097,  0.2842,  ...,  0.1281,  0.3838,  0.3247],
          [ 0.7250,  0.5849,  0.0344,  ...,  0.2448,  0.7588,  0.4986],
          [ 0.4457,  0.9543,  0.0584,  ...,  0.2401,  0.6713,  0.5698]]],
        device='cuda:0', grad_fn=<DropoutAddLayerNo

In [15]:
b = Block(1024, fused_dropout_add_ln=True).cuda()
b(torch.rand((1, 512, 1024), device="cuda"))

RuntimeError: FWD: Unsupported hidden_size or types: 1024FloatFloatFloatFloatFloat

In [3]:
dropout_layer_norm.dropout_add_ln_bwd()

['__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'dropout_add_ln_bwd',
 'dropout_add_ln_fwd',
 'dropout_add_ln_parallel_residual_bwd',
 'dropout_add_ln_parallel_residual_fwd']

In [ ]:
a = torch.tensor(0).to(dtype=torch.float16).cuda()


In [ ]:
from flash_attn.modules.block import Block
block = Block(64)

In [ ]:
sys.path.append("d:\\projects\\affect\\TUCORE-GCN\\tucore_gcn_transformers")
sys.path.append("d:\\projects\\affect\\TUCORE-GCN\\")

In [ ]:
from transformers.models.bert.modeling_bert import BertModel
from transformers.models.bert.configuration_bert import BertConfig
config = BertConfig.from_json_file("./models/BERT/tucoregcn_bert_mlc.json")
bertmodel = BertModel.from_pretrained("bert-base-uncased", config=config)

In [ ]:
for name, p in bertmodel.named_parameters():
	print(name)
	p.cuda()

: 

In [ ]:
bertmodel.cuda()

: 